In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# read the data
melb = pd.read_csv('../../data/melb_data.csv')

In [3]:
y = melb.Price
X = melb.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [4]:
# step 1
# define preprocessing steps

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [5]:
# preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

In [7]:
# preprocessing for categorical data
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])


In [8]:
# bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_cols),
                                               ('cat', categorical_transformer, categorical_cols)])

In [9]:
# step 2
# define the model

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [10]:
# step 3
# create and evaluate the pipeline

from sklearn.metrics import mean_absolute_error

# bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)])

# preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train)

# preprocessing of validation data, get prediction
preds = my_pipeline.predict(X_valid)

# evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 160679.18917034855


In [2]:
df_train = pd.read_csv('../../data/home-data-for-ml-course/train.csv', index_col='Id')

df_test = pd.read_csv('../../data/home-data-for-ml-course/test.csv', index_col='Id')

In [96]:
y = df_train.SalePrice
X = df_train.drop(['SalePrice'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 50 and X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [82]:
# step 1
# define preprocessing steps

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# preprocessing for categorical data
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_cols),
                                               ('cat', categorical_transformer, categorical_cols)])

In [83]:
from xgboost import XGBRegressor
# Define the model
my_model_2 = XGBRegressor(n_estimators=8000, learning_rate=0.001) # Your code here

In [84]:
# step 3
# create and evaluate the pipeline

from sklearn.metrics import mean_absolute_error

# bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', my_model_2)])

# Fit the model
my_pipeline.fit(X_train, 
               y_train)


# preprocessing of validation data, get prediction
preds = my_pipeline.predict(X_valid)

# evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 17373.944108518837


In [90]:
from sklearn.model_selection import cross_val_score

In [99]:
def get_score(n_estimators):
    """Return the average MAE over 3 CV folds of random forest model.
    
    Keyword argument:
    n_estimators -- the number of trees in the forest
    """
    # Replace this body with your own code
    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('model', XGBRegressor(n_estimators=n_estimators, learning_rate=0.01))])
    scores = -1 * cross_val_score(my_pipeline, 
                                  X_train, 
                                  y_train,
                                  cv=5,
                                  scoring='neg_mean_absolute_error')
    return scores.mean()

In [100]:
get_score(1000)

16495.18107360905